In [117]:
import pandas as pd
import numpy as np
import operator

from sklearn.feature_selection import mutual_info_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neural_network import MLPClassifier

from lightgbm import LGBMClassifier

In [2]:
df_transaction = catalog.load('train_transaction')
df_id = catalog.load('train_identity')

2022-04-24 12:44:16,016 - kedro.io.data_catalog - INFO - Loading data from `train_transaction` (CSVDataSet)...
2022-04-24 12:44:25,237 - kedro.io.data_catalog - INFO - Loading data from `train_identity` (CSVDataSet)...


In [26]:
df_id.columns

Index(['id_01', 'id_02', 'id_03', 'id_04', 'id_05', 'id_06', 'id_07', 'id_08',
       'id_09', 'id_10', 'id_11', 'id_12', 'id_13', 'id_14', 'id_15', 'id_16',
       'id_17', 'id_18', 'id_19', 'id_20', 'id_21', 'id_22', 'id_23', 'id_24',
       'id_25', 'id_26', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_32',
       'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo'],
      dtype='object')

In [5]:
df_join = df_transaction.merge(df_id, how='left', left_index=True, right_index=True)

In [29]:
# COLUMNS WITH STRINGS
str_type = ['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain','M1', 'M2', 'M3', 'M4','M5',
            'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType', 'DeviceInfo']
str_type += ['id_12', 'id_15', 'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 
            'id_31', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38']

# FIRST 53 COLUMNS
cols = ['TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9']

# V COLUMNS TO LOAD DECIDED BY CORRELATION EDA
# https://www.kaggle.com/cdeotte/eda-for-columns-v-and-id
v =  [1, 3, 4, 6, 8, 11]
v += [13, 14, 17, 20, 23, 26, 27, 30]
v += [36, 37, 40, 41, 44, 47, 48]
v += [54, 56, 59, 62, 65, 67, 68, 70]
v += [76, 78, 80, 82, 86, 88, 89, 91]

#v += [96, 98, 99, 104] #relates to groups, no NAN 
v += [107, 108, 111, 115, 117, 120, 121, 123] # maybe group, no NAN
v += [124, 127, 129, 130, 136] # relates to groups, no NAN

# LOTS OF NAN BELOW
v += [138, 139, 142, 147, 156, 162] #b1
v += [165, 160, 166] #b1
v += [178, 176, 173, 182] #b2
v += [187, 203, 205, 207, 215] #b2
v += [169, 171, 175, 180, 185, 188, 198, 210, 209] #b2
v += [218, 223, 224, 226, 228, 229, 235] #b3
v += [240, 258, 257, 253, 252, 260, 261] #b3
v += [264, 266, 267, 274, 277] #b3
v += [220, 221, 234, 238, 250, 271] #b3

v += [294, 284, 285, 286, 291, 297] # relates to grous, no NAN
v += [303, 305, 307, 309, 310, 320] # relates to groups, no NAN
v += [281, 283, 289, 296, 301, 314] # relates to groups, no NAN
#v += [332, 325, 335, 338] # b4 lots NAN

cols += ['V'+str(x) for x in v]
dtypes = {}
for c in cols+['id_0'+str(x) for x in range(1,10)]+['id_'+str(x) for x in range(10,34)]+\
    ['id_0'+str(x) for x in range(1,10)]+['id_'+str(x) for x in range(10,34)]:
        dtypes[c] = 'float32'
for c in str_type: dtypes[c] = 'category'

In [33]:
df_join = df_join.astype(dtypes)

In [34]:
df_join.head()

,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
TransactionID,,,,,,,,,,,,,,,,,,,,,
2987000,0,86400.0,68.5,W,13926.0,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987001,0,86401.0,29.0,W,2755.0,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987002,0,86469.0,59.0,W,4663.0,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987003,0,86499.0,50.0,W,18132.0,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2987004,0,86506.0,50.0,H,4497.0,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M


In [39]:
X = df_join.drop('isFraud', axis=1)
y = df_join['isFraud']

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.30)

In [45]:
model = LGBMClassifier(random_state=42, max_depth=7)

In [46]:
model.fit(X_train, y_train)

LGBMClassifier(max_depth=7, random_state=42)

In [48]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)

In [50]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    277732
           1       0.96      0.50      0.65     10096

    accuracy                           0.98    287828
   macro avg       0.97      0.75      0.82    287828
weighted avg       0.98      0.98      0.98    287828



In [51]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    118992
           1       0.92      0.47      0.62      4364

    accuracy                           0.98    123356
   macro avg       0.95      0.73      0.80    123356
weighted avg       0.98      0.98      0.98    123356



In [59]:
columns_half = int(len(df_join.columns[1:])/2)

In [62]:
columns1 = df_join.columns[1:columns_half]
columns2 = df_join.columns[columns_half:]

In [72]:
X_train_1 = X_train[columns1]
X_train_2 = X_train[columns2]

X_test_1 = X_test[columns1]
X_test_2 = X_test[columns2]

In [73]:
model_bank1 = LGBMClassifier(random_state=42)
model_bank2 = LGBMClassifier(random_state=42)

In [74]:
model_bank1.fit(X_train_1, y_train)
model_bank2.fit(X_train_2, y_train)

LGBMClassifier(random_state=42)

In [191]:
pred1 = model_bank1.predict_proba(X_train_1)
pred2 = model_bank2.predict_proba(X_train_2)

pred1_test = model_bank1.predict_proba(X_test_1)
pred2_test = model_bank2.predict_proba(X_test_2)

In [112]:
pred_final = []

for i in range(len(X_train_1)):
    pred_final.append(np.hstack((pred1[i], pred2[i])))

In [118]:
model_final = MLPClassifier(
    hidden_layer_sizes=(500, 100, 30), learning_rate_init=0.001, verbose=True
)

In [151]:
model_final.fit(pred_final, y_train.reset_index()['isFraud'])

Iteration 1, loss = 0.06927634
Iteration 2, loss = 0.06219373
Iteration 3, loss = 0.06178642
Iteration 4, loss = 0.06162904
Iteration 5, loss = 0.06152753
Iteration 6, loss = 0.06135160
Iteration 7, loss = 0.06122047
Iteration 8, loss = 0.06104491
Iteration 9, loss = 0.06088922
Iteration 10, loss = 0.06090634
Iteration 11, loss = 0.06086483
Iteration 12, loss = 0.06082754
Iteration 13, loss = 0.06073203
Iteration 14, loss = 0.06074999
Iteration 15, loss = 0.06073291
Iteration 16, loss = 0.06065957
Iteration 17, loss = 0.06062902
Iteration 18, loss = 0.06065417
Iteration 19, loss = 0.06063386
Iteration 20, loss = 0.06060440
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(hidden_layer_sizes=(500, 100, 30), verbose=True)

In [192]:
pred_final_test = []

for i in range(len(X_test_1)):
    pred_final_test.append(np.hstack((pred1_test[i], pred2_test[i])))

In [153]:
print(classification_report(y_test.reset_index()['isFraud'], model_final.predict(pred_final_test)))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97    118992
           1       0.04      0.02      0.03      4364

    accuracy                           0.94    123356
   macro avg       0.50      0.50      0.50    123356
weighted avg       0.93      0.94      0.94    123356



In [175]:
a = pd.DataFrame(pred_final)

In [176]:
a['target'] = y_train.reset_index()['isFraud']

In [177]:
X_f = a.drop('target', axis=1)
y_f = a['target']

In [178]:
model_final = MLPClassifier(
    hidden_layer_sizes=(500, 100, 10), learning_rate_init=0.001, verbose=True
)

In [179]:
model_final.fit(X_f, y_f)

Iteration 1, loss = 0.07024767
Iteration 2, loss = 0.06204459
Iteration 3, loss = 0.06193924
Iteration 4, loss = 0.06156993
Iteration 5, loss = 0.06143936
Iteration 6, loss = 0.06128142
Iteration 7, loss = 0.06111594
Iteration 8, loss = 0.06104650
Iteration 9, loss = 0.06101656
Iteration 10, loss = 0.06089691
Iteration 11, loss = 0.06086339
Iteration 12, loss = 0.06084877
Iteration 13, loss = 0.06073190
Iteration 14, loss = 0.06086313
Iteration 15, loss = 0.06075371
Iteration 16, loss = 0.06072179
Iteration 17, loss = 0.06064212
Iteration 18, loss = 0.06060605
Iteration 19, loss = 0.06064127
Iteration 20, loss = 0.06056813
Iteration 21, loss = 0.06060415
Iteration 22, loss = 0.06057753
Iteration 23, loss = 0.06050995
Iteration 24, loss = 0.06057839
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.


MLPClassifier(hidden_layer_sizes=(500, 100, 10), verbose=True)

In [180]:
print(classification_report(y_f, model_final.predict(X_f)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    277732
           1       0.87      0.60      0.71     10096

    accuracy                           0.98    287828
   macro avg       0.93      0.80      0.85    287828
weighted avg       0.98      0.98      0.98    287828



In [50]:
print(classification_report(y_train, y_pred_train))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    277732
           1       0.96      0.50      0.65     10096

    accuracy                           0.98    287828
   macro avg       0.97      0.75      0.82    287828
weighted avg       0.98      0.98      0.98    287828



In [193]:
b = pd.DataFrame(pred_final_test)
b['target'] = y_test.reset_index()['isFraud']

In [194]:
X_f1 = b.drop('target', axis=1)
y_f1 = b['target']

In [195]:
print(classification_report(y_f1, model_final2.predict(X_f1)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    118992
           1       0.84      0.55      0.67      4364

    accuracy                           0.98    123356
   macro avg       0.91      0.77      0.83    123356
weighted avg       0.98      0.98      0.98    123356



In [51]:
print(classification_report(y_test, y_pred_test))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    118992
           1       0.92      0.47      0.62      4364

    accuracy                           0.98    123356
   macro avg       0.95      0.73      0.80    123356
weighted avg       0.98      0.98      0.98    123356



In [196]:
np.unique(y_test, return_counts=True)

(array([0, 1]), array([118992,   4364]))

In [197]:
np.unique(y_f1, return_counts=True)

(array([0, 1]), array([118992,   4364]))

In [85]:
print(classification_report(y_test, model_bank1.predict(X_test_1)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    118992
           1       0.90      0.46      0.60      4364

    accuracy                           0.98    123356
   macro avg       0.94      0.73      0.80    123356
weighted avg       0.98      0.98      0.98    123356



In [86]:
print(classification_report(y_test, model_bank2.predict(X_test_2)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99    118992
           1       0.90      0.33      0.48      4364

    accuracy                           0.97    123356
   macro avg       0.94      0.66      0.73    123356
weighted avg       0.97      0.97      0.97    123356



In [4]:
df_train_transaction.isFraud.value_counts()[1]/df_train_transaction.shape[0]

0.03516673800536986

In [31]:
[0]*5+[1]*5

[0, 0, 0, 0, 0, 1, 1, 1, 1, 1]

In [29]:
df_train_transaction.select_dtypes(object)

,ProductCD,card4,card6,P_emaildomain,R_emaildomain,M1,M2,M3,M4,M5,M6,M7,M8,M9
0,W,discover,credit,NaN,NaN,T,T,T,M2,F,T,NaN,NaN,NaN
1,W,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN
2,W,visa,debit,outlook.com,NaN,T,T,T,M0,F,F,F,F,F
3,W,mastercard,debit,yahoo.com,NaN,NaN,NaN,NaN,M0,T,F,NaN,NaN,NaN
4,H,mastercard,credit,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411179,W,visa,debit,gmail.com,NaN,T,T,F,NaN,NaN,F,NaN,NaN,NaN
411180,H,visa,debit,aol.com,msn.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411181,H,visa,debit,gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
411182,W,visa,debit,NaN,NaN,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN


In [11]:
colunas = {}
for col in df_train_transaction.select_dtypes(include=[np.number]).columns:
    colunas[col] = mutual_info_classif(X=df_train_transaction[col].fillna(0).values.reshape(-1,1) ,y=df_train_transaction.isFraud, random_state=42)[0]

In [15]:
colunas_sort = sorted(colunas.items(), key=operator.itemgetter(1), reverse=True)